In [1]:
!pip install -q mediapipe
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.6 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [2]:
import os
import cv2
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np

In [3]:
# Constants
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # vibrant green
INPUT_FOLDER = "/kaggle/input/isl-words-dataset-reduced-sharpened/isl_words_dataset_reduced_sharpened"
OUTPUT_FOLDER = "/kaggle/working/isl_words_landmarks"

In [4]:
# Ensure output directory exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [5]:
def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

In [6]:
# Initialize the hand detector
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [7]:
def process_images(input_folder, output_folder):
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                input_path = os.path.join(root, file)
                rel_path = os.path.relpath(input_path, input_folder)
                output_path = os.path.join(output_folder, rel_path)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                image = mp.Image.create_from_file(input_path)
                detection_result = detector.detect(image)

                annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

                cv2.imwrite(output_path, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
                print(f"Processed and saved: {output_path}")

In [8]:
# Process all images in the dataset
process_images(INPUT_FOLDER, OUTPUT_FOLDER)

Processed and saved: /kaggle/working/isl_words_landmarks/small/623.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/208.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/187.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/682.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/489.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/824.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/768.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/82.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/334.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/829.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/630.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/120.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/512.jpg
Processed and saved: /kaggle/working/isl_words_landmarks/small/604.jpg
Process